In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sportsipy.ncaab.boxscore import Boxscore
from sportsipy.ncaab.boxscore import Boxscores
from sportsipy.ncaab.schedule import Schedule

In [2]:
games = Boxscores(datetime(2017,12,2), datetime(2017,12,2))

In [3]:
final = Boxscore('2017-12-02-15-duke').dataframe
final

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,home_two_point_field_goals,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name
2017-12-02-15-duke,60.0,18,0.0,0,121.5,48.3,14,0.515,68,0.441,...,28,1.0,10,"Cameron Indoor Stadium, Durham, North Carolina",SOUTH-DAKOTA,South Dakota,78.5,Home,DUKE,Duke


In [ ]:
for date in games.games.values():
    for game in date:
        final.append(Boxscore(game['boxscore']).dataframe,ignore_index = True)

In [4]:
from sportsipy.ncaab.teams import Teams

all_2019 = Teams(year=2019)

In [5]:
df2019 = all_2019.dataframes
df2019

,abbreviation,assist_percentage,assists,away_losses,away_wins,block_percentage,blocks,conference,conference_losses,conference_wins,...,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_percentage,wins
ABILENE-CHRISTIAN,ABILENE-CHRISTIAN,58.5,525,4,10,8.0,93,southland,4,14,...,1251,0.516,646,50.3,1110,0.565,15.5,407,0.794,27
AIR-FORCE,AIR-FORCE,54.1,434,9,3,5.8,57,mwc,10,8,...,1065,0.533,568,50.1,1077,0.541,17.4,423,0.438,14
AKRON,AKRON,50.1,399,10,1,8.9,106,mac,10,8,...,1019,0.491,500,48.2,1204,0.515,15.0,388,0.515,17
ALABAMA-AM,ALABAMA-AM,52.3,385,18,0,4.7,50,swac,14,4,...,1231,0.450,554,47.1,1032,0.479,19.4,487,0.156,5
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,44.3,401,6,6,7.5,82,cusa,8,10,...,1309,0.513,672,52.7,1279,0.536,14.8,399,0.571,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRIGHT-STATE,WRIGHT-STATE,54.7,484,8,5,6.1,72,horizon,5,13,...,1210,0.499,604,52.3,1229,0.543,14.6,402,0.600,21
WYOMING,WYOMING,48.0,331,11,1,7.9,88,mwc,14,4,...,936,0.472,442,44.7,983,0.534,18.6,450,0.250,8
XAVIER,XAVIER,56.3,519,8,4,10.6,128,big-east,9,9,...,1237,0.547,677,53.4,1281,0.553,16.5,450,0.543,19
YALE,YALE,56.3,503,5,8,11.2,131,ivy,4,10,...,1178,0.563,663,52.9,1157,0.584,15.9,392,0.733,22


In [6]:
df2019.index.tolist()

['ABILENE-CHRISTIAN',
 'AIR-FORCE',
 'AKRON',
 'ALABAMA-AM',
 'ALABAMA-BIRMINGHAM',
 'ALABAMA-STATE',
 'ALABAMA',
 'ALBANY-NY',
 'ALCORN-STATE',
 'AMERICAN',
 'APPALACHIAN-STATE',
 'ARIZONA-STATE',
 'ARIZONA',
 'ARKANSAS-LITTLE-ROCK',
 'ARKANSAS-PINE-BLUFF',
 'ARKANSAS-STATE',
 'ARKANSAS',
 'ARMY',
 'AUBURN',
 'AUSTIN-PEAY',
 'BALL-STATE',
 'BAYLOR',
 'BELMONT',
 'BETHUNE-COOKMAN',
 'BINGHAMTON',
 'BOISE-STATE',
 'BOSTON-COLLEGE',
 'BOSTON-UNIVERSITY',
 'BOWLING-GREEN-STATE',
 'BRADLEY',
 'BRIGHAM-YOUNG',
 'BROWN',
 'BRYANT',
 'BUCKNELL',
 'BUFFALO',
 'BUTLER',
 'CAL-POLY',
 'CAL-STATE-BAKERSFIELD',
 'CAL-STATE-FULLERTON',
 'CAL-STATE-NORTHRIDGE',
 'CALIFORNIA-BAPTIST',
 'CALIFORNIA-DAVIS',
 'CALIFORNIA-IRVINE',
 'CALIFORNIA-RIVERSIDE',
 'CALIFORNIA-SANTA-BARBARA',
 'CALIFORNIA',
 'CAMPBELL',
 'CANISIUS',
 'CENTRAL-ARKANSAS',
 'CENTRAL-CONNECTICUT-STATE',
 'CENTRAL-FLORIDA',
 'CENTRAL-MICHIGAN',
 'CHARLESTON-SOUTHERN',
 'CHARLOTTE',
 'CHATTANOOGA',
 'CHICAGO-STATE',
 'CINCINNATI',
 'CI

In [7]:
from sportsipy.ncaab.schedule import Schedule

duke_schedule = Schedule('Duke', 2018)

duke_schedule.dataframe_extended.head()

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,home_two_point_field_goals,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name
2017-11-10-19-duke,40.0,10,2.0,1,129.3,60.5,23,0.508,63,0.397,...,29,1.0,1,"Cameron Indoor Stadium, Durham, North Carolina",ELON,Elon,74.8,Home,DUKE,Duke
2017-11-11-19-duke,52.0,13,0.0,0,123.8,61.8,21,0.418,67,0.373,...,29,1.0,2,"Cameron Indoor Stadium, Durham, North Carolina",UTAH-VALLEY,Utah Valley,80.3,Home,DUKE,Duke
2017-11-14-19-michigan-state,70.0,21,13.9,5,103.8,72.4,21,0.454,76,0.395,...,22,0.5,1,"United Center, Chicago, Illinois",MICHIGAN-STATE,Michigan State,78.2,Away,DUKE,Duke
2017-11-17-19-duke,36.4,8,11.1,4,109.9,55.9,19,0.381,67,0.328,...,21,1.0,4,"Cameron Indoor Stadium, Durham, North Carolina",SOUTHERN,Southern,71.4,Home,DUKE,Duke
2017-11-20-19-duke,56.0,14,0.0,0,137.3,48.1,13,0.461,64,0.391,...,34,1.0,5,"Cameron Indoor Stadium, Durham, North Carolina",FURMAN,Furman,67.0,Home,DUKE,Duke


I opted to use and iterate through the **Schedule** class for 3 reasons:
1. It functions as a 'sort by' per team.
2. You can search by year. That's cumbersome so I'm iterating in a for loop.
3. It can produce a class target output (see *winning_abbr* in DF above)
It also provides usable stats that we can calculate over a season to calculate things like mean FG pct, FG attempts, etc.

In [20]:
from sportsipy.ncaab.teams import Teams
from sportsipy.ncaab.schedule import Schedule

# This creates a list of all the teams that we can use to iterate through
# in the function
team_names = Teams(year=2020)
team_list = team_names.dataframes
team_names_list = team_list.index.tolist()

# This is a list of the years we want the team data for, also used to iterate
# through in the function
year_range_list = list(range(2019, 2022))

# The function takes in the list of teams and year range. I looped through each
# year and used the iteration as the year input in the Schedule() class function.
# I tried to create a DataFrame every time that iteration happened, append it to
# an empty DataFrame and return the final concatenated DataFrame.
def team_schedule(teams_list, year_range):
    for year in year_range:
        final_df = pd.DataFrame()
        for team in teams_list:
            team_sched = Schedule(team, year)
            df_to_append = team_sched.dataframe_extended
            pd.concat([final_df, df_to_append], ignore_index=True)
    return final_df

In [ ]:
team_schedule(team_names_list, year_range_list)